In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy.sparse.linalg import svds
from flask import Flask, render_template, request
import pickle


In [2]:
book_df = pd.read_csv('Books.csv')
ratings_df = pd.read_csv('Ratings.csv').sample(40000)
user_df = pd.read_csv('Users.csv')

user_rating_df = ratings_df.merge(user_df, left_on = 'User-ID', right_on = 'User-ID')
book_user_rating = book_df.merge(user_rating_df, left_on = 'ISBN',right_on = 'ISBN')
book_user_rating = book_user_rating[['ISBN', 'Book-Title', 'Book-Author', 'User-ID', 'Book-Rating']]
book_user_rating.reset_index(drop=True, inplace = True)

d ={}
for i,j in enumerate(book_user_rating.ISBN.unique()):
    d[j] =i
book_user_rating['unique_id_book'] = book_user_rating['ISBN'].map(d)

users_books_pivot_matrix_df = book_user_rating.pivot(index='User-ID', 
                                                              columns='unique_id_book', 
                                                              values='Book-Rating').fillna(0)
users_books_pivot_matrix_df = users_books_pivot_matrix_df.values

NUMBER_OF_FACTORS_MF = 15

    #Performs matrix factorization of the original user item matrix
U, sigma, Vt = svds(users_books_pivot_matrix_df, k = NUMBER_OF_FACTORS_MF)

sigma = np.diag(sigma)
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) 


C:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [63]:
book_user_rating.to_pickle("book_user_rating.pkl")
pickle.dump(d, open("d.p", "wb"))  # save it into a file named save.p
with open('Vt','wb') as f: pickle.dump(Vt, f)



In [78]:
def top_cosine_similarity(data, book_id, top_n=10):
    index = book_id 
    book_row = data[index, :]
    magnitude = np.sqrt(np.einsum('ij, ij -> i', data, data))
    similarity = np.dot(book_row, data.T) / (magnitude[index] * magnitude)
    sort_indexes = np.argsort(-similarity)
    return sort_indexes[:top_n]

def similar_books(book_user_rating, book_id, top_indexes):
    l = []
    print('Recommendations for {0}: \n'.format(
    book_user_rating[book_user_rating.unique_id_book == book_id]['Book-Title'].values[0]))
    for id in top_indexes + 1:
        l.append(book_user_rating[book_user_rating.unique_id_book == id]['Book-Title'].values[0])
    return l
        
movie_id =25954  

similar_books(book_user_rating, 25954, top_cosine_similarity(Vt.T[:, :50], movie_id, 3))

Recommendations for Winners, Losers &amp; Microsoft: Competition and Antitrust in High Technology: 



<ipython-input-78-bada402575f6>:5: RuntimeWarning: invalid value encountered in true_divide
  similarity = np.dot(book_row, data.T) / (magnitude[index] * magnitude)


['Bloody Jack: Being an Account of the Curious Adventures of Mary \\Jacky\\" Faber, Ship\'s Boy"',
 'Scottish Crofters:  A Historical Ethnography of a Celtic Village',
 'Beneath the Skin']

In [69]:
import marshal, types

top_cosine_similarity_string = marshal.dumps(top_cosine_similarity.__code__)
topsimilarity_code = marshal.loads(top_cosine_similarity_string)
top_cosine_similarity = types.FunctionType(topsimilarity_code, globals(), "some_func_name")

similar_books_string = marshal.dumps(similar_books.__code__)
similar_bookg_code = marshal.loads(similar_books_string)
similar_books = types.FunctionType(similar_bookg_code, globals(), "some_func_name")


similar_books(book_user_rating, 25954, top_cosine_similarity(Vt.T[:, :50], movie_id, 3))

Recommendations for Winners, Losers &amp; Microsoft: Competition and Antitrust in High Technology: 

Bloody Jack: Being an Account of the Curious Adventures of Mary \Jacky\" Faber, Ship's Boy"
Scottish Crofters:  A Historical Ethnography of a Celtic Village
Beneath the Skin


<ipython-input-64-21fa7d45ac8f>:5: RuntimeWarning: invalid value encountered in true_divide
  similarity = np.dot(book_row, data.T) / (magnitude[index] * magnitude)


In [14]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy.sparse.linalg import svds
from flask import Flask, render_template, request
import pickle

d = pickle.load(open("d.p", "rb"))
with open('Vt','rb') as f: Vt = pickle.load(f)
book_user_rating =  pickle.load(open("book_user_rating.pkl", "rb"))
np.seterr(divide='ignore', invalid='ignore')


def top_cosine_similarity(data, book_id, top_n=10):
    index = book_id 
    book_row = data[index, :]
    magnitude = np.sqrt(np.einsum('ij, ij -> i', data, data))
    similarity = np.dot(book_row, data.T) / (magnitude[index] * magnitude)
    sort_indexes = np.argsort(-similarity)
    return sort_indexes[:top_n]

def similar_books(book_user_rating, book_id, top_indexes):
    l = []
    print('Recommendations for {0}: \n'.format(
    book_user_rating[book_user_rating.unique_id_book == book_id]['Book-Title'].values[0]))
    for id in top_indexes + 1:
        l.append(book_user_rating[book_user_rating.unique_id_book == id]['Book-Title'].values[0])
    return l


app = Flask(__name__)

@app.route('/')
def home():
    return render_template('home.html')

@app.route('/predict',methods=['POST'])
def predict():

    if request.method == 'POST':
        message = request.form['message']
        index = top_cosine_similarity(Vt.T[:, :50], int(message) , 3)
        
        my_prediction = similar_books(book_user_rating,int(message) ,index)
        s= {}
     
        for i in range(len(my_prediction)):
            s[i+1]= my_prediction[i]
       
 

        return render_template('result.html',prediction = s)



if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Nov/2021 16:58:20] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Nov/2021 16:58:20] "GET /static/css/styles.css HTTP/1.1" 404 -
<ipython-input-14-5ae48edb07d5>:17: RuntimeWarning: invalid value encountered in true_divide
  similarity = np.dot(book_row, data.T) / (magnitude[index] * magnitude)
127.0.0.1 - - [03/Nov/2021 16:58:27] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Nov/2021 16:58:27] "GET /static/css/styles.css HTTP/1.1" 404 -


Recommendations for Thrill: 



In [4]:
def top_cosine_similarity(data, book_id, top_n=10):
    index = book_id 
    book_row = data[index, :]
    magnitude = np.sqrt(np.einsum('ij, ij -> i', data, data))
    similarity = np.dot(book_row, data.T) / (magnitude[index] * magnitude)
    sort_indexes = np.argsort(-similarity)
    return sort_indexes[:top_n]

def similar_books(book_user_rating, book_id, top_indexes):
    l = []
    print('Recommendations for {0}: \n'.format(
    book_user_rating[book_user_rating.unique_id_book == book_id]['Book-Title'].values[0]))
    for id in top_indexes + 1:
        l.append(book_user_rating[book_user_rating.unique_id_book == id]['Book-Title'].values[0])
    return l

In [8]:
d = pickle.load(open("d.p", "rb"))
with open('Vt','rb') as f: Vt = pickle.load(f)
book_user_rating =  pickle.load(open("book_user_rating.pkl", "rb"))
        
def top_cosine_similarity(data, book_id, top_n=10):
    index = book_id 
    book_row = data[index, :]
    magnitude = np.sqrt(np.einsum('ij, ij -> i', data, data))
    similarity = np.dot(book_row, data.T) / (magnitude[index] * magnitude)
    sort_indexes = np.argsort(-similarity)
    return sort_indexes[:top_n]

def similar_books(book_user_rating, book_id, top_indexes):
    l = []
    print('Recommendations for {0}: \n'.format(
    book_user_rating[book_user_rating.unique_id_book == book_id]['Book-Title'].values[0]))
    for id in top_indexes + 1:
        l.append(book_user_rating[book_user_rating.unique_id_book == id]['Book-Title'].values[0])
    return l
np.seterr(divide='ignore', invalid='ignore')

index = top_cosine_similarity(Vt.T[:, :50], 123, 3)
index

array([  123,  3009, 21781], dtype=int64)

In [7]:
book_user_rating =  pickle.load(open("book_user_rating.pkl", "rb"))
max(book_user_rating['unique_id_book'].values)

26034

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy.sparse.linalg import svds
from flask import Flask, render_template, request
import pickle

d = pickle.load(open("d.p", "rb"))
with open('Vt','rb') as f: Vt = pickle.load(f)
book_user_rating =  pickle.load(open("book_user_rating.pkl", "rb"))
np.seterr(divide='ignore', invalid='ignore')


def top_cosine_similarity(data, book_id, top_n=10):
    index = book_id 
    book_row = data[index, :]
    magnitude = np.sqrt(np.einsum('ij, ij -> i', data, data))
    similarity = np.dot(book_row, data.T) / (magnitude[index] * magnitude)
    sort_indexes = np.argsort(-similarity)
    return sort_indexes[:top_n]

def similar_books(book_user_rating, book_id, top_indexes):
    l = []
    print('Recommendations for {0}: \n'.format(
    book_user_rating[book_user_rating.unique_id_book == book_id]['Book-Title'].values[0]))
    for id in top_indexes + 1:
        l.append(book_user_rating[book_user_rating.unique_id_book == id]['Book-Title'].values[0])
    return l


app = Flask(__name__)



@app.route('/')
def home():
    return render_template('home.html')

@app.route('/predict',methods=['POST'])
def predict():

    if request.method == 'POST':
        try:
            message = request.form['message']       
            index = top_cosine_similarity(Vt.T[:, :50], int(message) , 3)

            my_prediction = similar_books(book_user_rating,int(message) ,index)
            s= {}

            for i in range(len(my_prediction)):
                s[i+1]= my_prediction[i]
                
            return render_template('result.html',prediction = s)
        except:
            return render_template('result.html',prediction = {'0': 'Invalid ISBN number, enter correct ISBN please! '})

if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Nov/2021 17:00:56] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Nov/2021 17:00:56] "GET /static/css/styles.css HTTP/1.1" 404 -
<ipython-input-22-fde328550a31>:17: RuntimeWarning: invalid value encountered in true_divide
  similarity = np.dot(book_row, data.T) / (magnitude[index] * magnitude)
127.0.0.1 - - [03/Nov/2021 17:00:58] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Nov/2021 17:00:58] "GET /static/css/styles.css HTTP/1.1" 404 -


Recommendations for Nights Below Station Street: 



127.0.0.1 - - [03/Nov/2021 17:00:59] "GET /static/css/styles.css HTTP/1.1" 404 -
127.0.0.1 - - [03/Nov/2021 17:01:05] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Nov/2021 17:01:05] "GET /static/css/styles.css HTTP/1.1" 404 -
